In [19]:
state= {
    "user_query": "Book my air ticket from Delhi to Mumbai",
    "chat_history": [
        "Marvin: Hi ! I am Marvin a CX Autonomous Agent for your day to day banking queries. May I have your name , email_ID and Password to start with?",
        "User: James jms@gmail.com 54132",
        "Marvin: Thank you James. How can I assist you today with your recent bookings or new ticket bookings?",
        "User: Can I know my bookings",
        "Marvin: Your last booking was from Jaipur to Delhi.",
        "User: I want to book my air ticket from Delhi to Mumbai",
        "Marvin: Ok Are you sure you want me to proceed?",
        "User: Yes",
        "Marvin: Would you like me to book a flight/train/bus?",
        "User: I want to book a flight",
        "Marvin: How many seats do you require?",
        "User: I want to book 3 seats",
        "Marvin: Understood. Your total would be Rs.14500.",
        "User: OK",
        "Marvin: Would you like me to process the payment?",
        "User: Yes. Sure!",
        "Marvin: Guiding to Payment Gateway.......Booking Successful",
        "Marvin: Based on your preferences, here's a customized itinerary for your trip to Paris: Day 1: Explore the Eiffel Tower and Seine River cruise. Day 2: Visit the Louvre Museum and Notre-Dame Cathedral. Day 3: Day trip to Versailles. Day 4: Discover Montmartre and Sacré-Cœur. Day 5: Shopping at Champs-Élysées and relax at Luxembourg Gardens.",
        "Marvin: Based on your preference for a beachfront hotel in Bali with a budget of $150 per night, I suggest staying at the Bali Beach Resort. It offers stunning views, free breakfast, and a spa.",
        "Marvin: Based on your interests in beaches and a budget of $2000, I suggest visiting Bali in June. It's a beautiful destination with stunning beaches and affordable accommodations."
    ],
    "booking_ref_number": "1234326890",
    "name": "James",
    "email_id": "jms@gmail.com",
    "account_id": 12,
    "password": "54132",
    "verification": True,
    "node_data": {},
    "tool": {
        "info": {
            "name": "transactions",
            "parameters": {
                "query": "SELECT Reason FROM Transactions WHERE account_id=12 AND Status=0 ORDER BY date DESC LIMIT 1"
            }
        },
        "agent": "Recent_transactions"
    },
    "tool_response": [
        {
            "Reason": "card limit was exceeded"
        }
    ],
    "previous_process_state": "Credit_limit",
    "response": "Understood. Your credit limit will not be changed.",
    "customer_account_number": "1234326890",
    "comment": "Verification successful. Customer and Account details:"
}


In [20]:
verification_prompt='''
You are the verification block of chatbot Marvin, your task is to gather information about the user's name, email ID, and password, and verify the information using the provided tool.
If some information is missing, then you should ask follow-up questions.
Marvin is a travel bot that helps customers with their travel bookings and related inquiries.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Today's Date: <today's date>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "verification",
        "description": "Verifies the user's identity against travel records.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The user's full name."
                },
                "email_id": {
                    "type": "string",
                    "description": "The user's email ID."
                },
                "password": {
                    "type": "string",
                    "description": "The user's password."
                }
            },
            "required": ["name", "email_id", "password"]
        }
    }
]
# Output Format:
Return a JSON object with the following structure. If multiple actions are needed, include them in a list, in the order they should be executed:
[
 {
     "Action": <either Response or Tool call as per the output>,
     "attribute": <in case of response return the string and in case of Tool call make it as a JSON>,
     "extracted_entity": {
                "name": "<user's full name>",
                "email_id": "<user's email ID>",
                "password": "<user's password>"
            } -- fill this placeholder of extracted_entity once you receive any data from user_input else return blank
    }
]

Example of different types of outputs:
1. [{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>",
        "extracted_entity": {
                "name": "<user's full name>",
                "email_id": "<user's email ID>",
                "password": "<user's password>"
            }
}]
2. [{
    "Action": "Tool call", // Use "Tool call" for calling the verification tool.
    "attribute": {
        "name": "verification",
        "parameters": {
            "name": "<user's full name>",
            "email_id": "<user's email ID>",
            "password": "<user's password>"
        }
    }
 "extracted_entity": {
                "name": "<user's full name>",
                "email_id": "<user's email ID>",
                "password": "<user's password>"
            }
}]

Note- only respond in JSON output
    - don't explicitly ask the user to provide the email ID or password in a certain format

# '''


In [21]:
verification_prompt_input='''
user_input:
{user_input}

chat_history:
{chat_history}

Previous Extracted entity:
{extracted_entity}

Today's Date:
2025-02-24
'''

In [22]:
Output = {
    "Action": "Response",
    "attribute": "Thank you James. How can I assist you today with your recent bookings or new ticket bookings?",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132"
    }
}

In [48]:
verification_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the verification block of chatbot Marvin, your task is to gather information about the user's name, email ID, and password, and verify the information using the provided tool. If some information is missing, then you should ask follow-up questions."
        },
        {
            "role": "user",
            "content": "<user_input>"
        },
        {
            "role": "verify_user_details",
            "parameters": {
                "name": "<user's full name>",
                "email": "<user's email ID>",
                "password": "<user's password>"
            }
            "content": {
            "Action": "Response",
            "attribute": "Thank you James. You are logged in now! How can I assist you today with your recent bookings or new ticket bookings?"
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Thank you James. How can I assist you today with your recent bookings or new ticket bookings?",
        "extracted_entity": {
            "name": "James",
            "email_id": "jms@gmail.com",
            "password": "54132"
        }
    }
}
'''


In [43]:
destination_suggestions_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the destination suggestions block of chatbot Marvin. Your task is to suggest travel destinations based on the user's preferences. Gather information about the user's interests, budget, and preferred travel dates. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "<user_input>Please suggest me with some destinations."
        },
        {
            "role": "destination_preferences",
            "parameters": {
                "interests": "",
                "budget": "",
                "travel_dates": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Based on your interests in beaches and a budget of $2000, I suggest visiting Bali in June. It's a beautiful destination with stunning beaches and affordable accommodations."
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Based on your interests in beaches and a budget of $2000, I suggest visiting Bali in June. It's a beautiful destination with stunning beaches and affordable accommodations.",
        "extracted_entity": {
            "interests": "beaches",
            "budget": "$2000",
            "travel_dates": "June"
        }
    }
}
'''


In [44]:
hotel_suggestions_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the hotel suggestions block of chatbot Marvin. Your task is to suggest hotels based on the user's preferences. Gather information about the user's destination, budget, and preferred amenities. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "<user_input>Please suggest me with some hotels."
        },
        {
            "role": "hotel_preferences",
            "parameters": {
                "destination": "",
                "budget": "",
                "amenities": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Based on your preference for a beachfront hotel in Bali with a budget of $150 per night, I suggest staying at the Bali Beach Resort. It offers stunning views, free breakfast, and a spa."
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Based on your preference for a beachfront hotel in Bali with a budget of $150 per night, I suggest staying at the Bali Beach Resort. It offers stunning views, free breakfast, and a spa.",
        "extracted_entity": {
            "destination": "Bali",
            "budget": "$150 per night",
            "amenities": "beachfront, free breakfast, spa"
        }
    }
}
'''


In [26]:
custom_itinerary_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the itinerary planning block of chatbot Marvin. Your task is to create a detailed travel itinerary based on the user's destination and preferences. Gather information about the user's destination, interests, budget, preferred activities, and the number of days for the trip. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "<user_input>Prepare an itinerary for my vacation"
        },
        {
            "role": "itinerary_preferences",
            "parameters": {
                "destination": "",
                "interests": "",
                "budget": "",
                "preferred_activities": "",
                "number_of_days": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Based on your preferences, here's a customized itinerary for your trip to Paris: Day 1: Explore the Eiffel Tower and Seine River cruise. Day 2: Visit the Louvre Museum and Notre-Dame Cathedral. Day 3: Day trip to Versailles. Day 4: Discover Montmartre and Sacré-Cœur. Day 5: Shopping at Champs-Élysées and relax at Luxembourg Gardens."
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Based on your preferences, here's a customized itinerary for your trip to Paris: Day 1: Explore the Eiffel Tower and Seine River cruise. Day 2: Visit the Louvre Museum and Notre-Dame Cathedral. Day 3: Day trip to Versailles. Day 4: Discover Montmartre and Sacré-Cœur. Day 5: Shopping at Champs-Élysées and relax at Luxembourg Gardens.",
        "extracted_entity": {
            "destination": "Paris",
            "interests": "sightseeing, culture",
            "budget": "$3000",
            "preferred_activities": "museums, landmarks, shopping",
            "number_of_days": "5"
        }
    }
}
'''


In [42]:
mode_selection_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the mode selection block of chatbot Marvin. Your task is to help the user select their preferred mode of travel. Gather information about the user's destination, travel dates, and preferences for train, air, or bus travel. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "I want to book my travel."
        },
        {
            "role": "travel_preferences",
            "parameters": {
                "destination": "",
                "travel_dates": "",
                "preferred_mode": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Based on your preference for air travel to Mumbai on June 15th, I suggest booking a flight with XYZ Airlines. They offer competitive prices and excellent service."
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Based on your preference for air travel to Mumbai on June 15th, I suggest booking a flight with XYZ Airlines. They offer competitive prices and excellent service.",
        "extracted_entity": {
            "destination": "Mumbai",
            "travel_dates": "June 15th",
            "preferred_mode": "air"
        }
    }
}
'''


In [45]:
ticket_booking_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the ticket booking block of chatbot Marvin. Your task is to assist the user in booking their travel tickets after they have selected their preferred mode of travel. Gather information about the user's destination, travel dates, number of passengers, and any specific preferences. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "I want to book a flight to Mumbai."
        },
        {
            "role": "booking_details",
            "parameters": {
                "destination": "Mumbai",
                "travel_dates": "",
                "number_of_passengers": "",
                "specific_preferences": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Please provide your travel dates and the number of passengers."
            }
        },
        {
            "role": "user",
            "content": "Travel date is June 15th and I need 3 seats."
        },
        {
            "role": "booking_details",
            "parameters": {
                "destination": "Mumbai",
                "travel_dates": "June 15th",
                "number_of_passengers": "3",
                "specific_preferences": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Based on your preferences, I have found a flight with XYZ Airlines on June 15th for 3 passengers. The total cost is Rs. 14500. Would you like to proceed with the booking?"
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Based on your preferences, I have found a flight with XYZ Airlines on June 15th for 3 passengers. The total cost is Rs. 14500. Would you like to proceed with the booking?",
        "extracted_entity": {
            "destination": "Mumbai",
            "travel_dates": "June 15th",
            "number_of_passengers": "3",
            "specific_preferences": ""
        }
    }
}
'''


In [27]:
process_identification_prompt_v1='''
You are Process identification block of Marvin, a travel chatbot assistant. Your role is to identify the user's intent and guide the conversation towards one of five supported travel processes:

1.  **`Air_travel`:** The user wants to book, cancel, or check the status of an air travel ticket.
2.  **`Train_travel`:** The user wants to book, cancel, or check the status of a train travel ticket.
3.  **`Bus_travel`:** The user wants to book, cancel, or check the status of a bus travel ticket.
4.  **`Payment_and_support`:** The user has issues related to payment methods, refunds, billing inquiries, or general customer support queries.
5.  **`Other`:** The user's intent does not clearly fall into one of the above categories.

For the "Other" intent, generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported travel topics.

**Input:**

*   **`user_input`:** The user's most recent message.
*   **`chat_history`:** The full conversation history between Marvin and the agent.
*   **`previous_process_state`:** The state of any previously identified process. This can be `null` (or not present) if no process has been started, or it could be a value representing a stage within `Air_travel`, `Train_travel`, `Bus_travel`, or `Payment_and_support` (you don't need to *interpret* these states, just pass them through; a separate module would handle state management).

**Output Format:**

Return a JSON object *without* wrapping it in a list. The JSON object *must* have the following structure:

{
    "Action": "<Intent>",
    "attribute": "<Response or null>"
}

Note- only respond in JSON output

'''

process_identification_prompt_v1_input='''
user_input:
{user_input}

chat_history
{chat_history}

previous_process_state:
{previous_process_state}
'''

Output = {
    "Action": "Response",
    "attribute": "Thank you James. How can I assist you today? Are you looking to travel by air, train, or bus?",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}

In [28]:
process_identification_prompt_v2='''
You are Process identification block of Marvin, a travel chatbot assistant. Your role is to identify the user's intent and guide the conversation towards one of three supported ticket booking processes:

1.  **`Book_air_ticket`:** The user wants to book an air travel ticket.
2.  **`Book_train_ticket`:** The user wants to book a train travel ticket.
3.  **`Book_bus_ticket`:** The user wants to book a bus travel ticket.

If the user's intent does not clearly fall into one of these three categories, classify it as **`Other`**. For the "Other" intent, generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported ticket booking topics.

**Input:**

*   **`user_input`:** The user's most recent message.
*   **`chat_history`:** The full conversation history between Marvin and the agent.
*   **`previous_process_state`:** The state of any previously identified process. This can be `null` (or not present) if no process has been started, or it could be a value representing a stage within `Book_air_ticket`, `Book_train_ticket`, or `Book_bus_ticket` (you don't need to *interpret* these states, just pass them through; a separate module would handle state management).

**Output Format:**

Return a JSON object *without* wrapping it in a list. The JSON object *must* have the following structure:

{
    "Action": "<Intent>",
    "attribute": "<Response or null>"
}

Note- only respond in JSON output

'''

process_identification_prompt_v2_input='''
user_input:
{user_input}

chat_history
{chat_history}

previous_process_state:
{previous_process_state}
'''

Output = {
    "Action": "Response",
    "attribute": "Thank you James. Can I proceed with your ticket booking?",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}

In [29]:

process_identification_prompt_v3='''
You are Process identification block of Marvin, a travel chatbot assistant. Your role is to identify the user's intent and guide the conversation towards one of three supported ticket cancellation processes:

1.  **`Cancel_air_ticket`:** The user wants to cancel an air travel ticket.
2.  **`Cancel_train_ticket`:** The user wants to cancel a train travel ticket.
3.  **`Cancel_bus_ticket`:** The user wants to cancel a bus travel ticket.

If the user's intent does not clearly fall into one of these three categories, classify it as **`Other`**. For the "Other" intent, generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported ticket cancellation topics.

**Input:**

*   **`user_input`:** The user's most recent message.
*   **`chat_history`:** The full conversation history between Marvin and the agent.
*   **`previous_process_state`:** The state of any previously identified process. This can be `null` (or not present) if no process has been started, or it could be a value representing a stage within `Cancel_air_ticket`, `Cancel_train_ticket`, or `Cancel_bus_ticket` (you don't need to *interpret* these states, just pass them through; a separate module would handle state management).

**Output Format:**

Return a JSON object *without* wrapping it in a list. The JSON object *must* have the following structure:

{
    "Action": "<Intent>",
    "attribute": "<Response or null>"
}

Note- only respond in JSON output

'''

process_identification_prompt_v3_input='''
user_input:
{user_input}

chat_history
{chat_history}

previous_process_state:
{previous_process_state}
'''

Output = {
    "Action": "Response",
    "attribute": "Thank you James. Can I proceed with your ticket cancellation?",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}

In [30]:
customer_support_and_payment_prompt='''
You are Customer Support and Payment Issues block of Marvin, a travel chatbot assistant. Your role is to handle user queries related to payment methods, refunds, billing inquiries, and general customer support. 

**Input:**

*   **`user_input`:** The user's most recent message.
*   **`chat_history`:** The full conversation history between Marvin and the user.

**Your Task:**

Based on these inputs, provide a clear, friendly, and helpful response to the user's query. Your response should:

*   **Address Payment Issues:** If the query is related to payment methods, refunds, or billing inquiries, provide relevant information or steps to resolve the issue.
*   **Provide Customer Support:** If the query is a general customer support question, offer assistance or direct the user to the appropriate resources.
*   **Be User-Friendly:** Avoid technical jargon and explain the information in a way that a non-technical user would easily understand.
*   **Use Chat History:** If chat history is provided, use it to avoid repeating information already provided and to provide more specific answers based on previous clarifications.

# Output Format:
Return a JSON object with the following structure:

{
    "Action": "Response",
    "attribute": "<Response generated>"
}

Note- only respond in JSON output

'''

pcustomer_support_and_paymen_input='''
user_input:
{user_input}

chat_history
{chat_history}

previous_process_state:
{previous_process_state}
'''

Output = {
    "Action": "Response",
    "attribute": "Sure James! Your refund will take 24hrs to reflect , its already processed from our system?",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}


In [31]:
transactions_prompt='''
You are a helpful SQL query generator for a travel database. You will be given user queries related to travel bookings, and you must generate valid SQL queries to answer those questions using the provided `Bookings` table schema. The `user_id` associated with the user's account is already known and available as a variable. You do *not* need to join other tables. Focus *only* on using the `Bookings` table.

**Bookings Table Schema:**

```sql
CREATE TABLE Bookings (
  booking_id INT PRIMARY KEY AUTO_INCREMENT,
  date DATETIME NOT NULL,  -- Date and time of the booking
  amount DECIMAL(19, 4) NOT NULL,  -- Booking amount
  type VARCHAR(50),  -- e.g., 'Flight', 'Hotel', 'Car Rental'
  vendor VARCHAR(255),  -- Vendor/Service provider for the booking
  status BOOLEAN,  -- Booking status (TRUE for confirmed, FALSE for cancelled)
                 -- (Consider boolean as 1 for True and 0 for False.)
  user_id INT,  -- Foreign key referencing the Users table
  FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE
);

# Output Format :
Return a JSON object with the following structure :

 {
     "Action":<either Response or Tool call as per the output (Note- Tool call is for sql statement)>,
     "attribute":< in case of response return the string and in case of Tool call make it as a JSON>
    }


Example of different type of outputs:
1.{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>"
		
}
2.{
	"Action": "Tool call", // Use "Tool call" for calling the bookings tool.
	"attribute": {
		"name": "bookings",
		"parameters": {
			"query": "<generated sql statement>",
			
		}
	}
 
}

Note- only respond in JSON output

'''

transactions_prompt_input='''
user_input:
{user_input}

chat_history
{chat_history}

MySQL Query Result
{result}

Today's Date:
2025-02-24
'''

Output = {
    "Action": "Response",
    "attribute": "response_recent_transaction ",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}

In [32]:

response_recent_transaction='''
You are response generator block of Marvin, a travel chatbot assistant. Your role is to generate responses for users in clear, friendly, and easy-to-understand language using the given input:

1.  **MySQL Query Result:** The result of a SQL query executed against the `Bookings` table, in a list-of-dictionaries format. Each dictionary represents a single booking.
2.  **Original User Query:** The user's initial question that led to the SQL query.
3.  **Chat History:** Previous turns in the conversation, if any, which can provide context. This will be a list of dictionaries, where each dictionary has keys "user" and "assistant".

**Your Task:**

Based on these inputs, provide a concise and informative response to the user's original query. Your response should:

*   **Be User-Friendly:** Avoid technical jargon. Explain the results in a way that a non-technical user would easily understand.
*   **Address the User's Query Directly:** Refer back to the original question and make it clear how the information provided answers it.
*   **Summarize, Don't Just List:** If there are multiple bookings in the result, synthesize the information. For example, instead of listing every booking, you might say: "You have three bookings with XYZ Airlines on July 7th, totaling $1500." If there's only one, explain that booking clearly.
*   **Handle Empty Results:** If the MySQL query result is an empty list (`[]`), inform the user that no matching bookings were found. Be polite and suggest possible reasons or alternative queries, if appropriate.
*   **Format Dates and Amounts:** Present dates and amounts in a user-friendly format (e.g., "July 7, 2024" instead of "2024-07-07 09:45:00", and "$750.00" instead of "750.0000").
*   **Interpret Status:** Explain the `Status` field in plain English (e.g., "This booking is confirmed" or "This booking was cancelled"). Recall that Status likely equates to 1 for confirmed, and 0 for cancelled.
* **Use Chat History:** If chat history is provided, use it to:
    *   Avoid repeating information already provided.
    *   Provide more specific answers based on previous clarifications. For example, if a previous turn clarified that "recent" means "last week," use that definition.
    *   Refer back to previous follow-up questions and their answers.
* **Consider implicit question**: If the user is asking a question like why was booking cancelled, consider answering the Reason field in the response.

# Output Format:
Return a JSON object with the following structure:

{
     "Action":"Response",
     "attribute":< Response generated >
}

Note- only respond in JSON output
'''

response_recent_transaction_input='''
user_input:
{user_input}

chat_history
{chat_history}

MySQL Query Result
{result}

Today's Date:
2025-02-24
'''

Output = {
    "Action": "Response",
    "attribute": "response_verification",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}


In [33]:
response_verification='''
You are response generator block of Marvin, a travel chatbot assistant. Your role is to generate responses for users in clear, friendly, and easy-to-understand language using the given input:

1.  **MySQL Query Result:** The result of a SQL query executed against the `verification` table.
2.  **Original User Query:** The user's initial question that led to the SQL query.
3.  **Chat History:** Previous turns in the conversation, if any, which can provide context. This will be a list of dictionaries, where each dictionary has keys "user" and "Marvin".

Rules:
1. If verification failed, then respond to the user in a gentle way that their verification failed and you are transferring them to an agent.
2. If verification is successful, then thank the user and ask how you can help with ticket booking, ticket cancellation, or ticket checking. Don't ask the user to provide any other user information.

# Output Format:
Return a JSON object with the following structure:

{
     "Action":"Response",
     "attribute":< Response generated >
}

Note- only respond in JSON output
'''

response_verification_input='''
user_input:
{user_input}

chat_history
{chat_history}

MySQL Query Result
{result}

Today's Date:
2025-02-24
'''

Output = {
    "Action": "Response",
    "attribute": " ",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": "54132",
        "mode": ""
    }
}


In [34]:
prompt_stack={
	"verification":{
		"system":verification_prompt,
		"input":verification_prompt_input}
}

In [35]:
messages = [{'role': 'system',
                     'content': prompt_stack['verification']['system']},
                    {'role': 'user',
                     'content': prompt_stack['verification']['input'].format(user_input=state['user_query'],
                                                                             chat_history="\n".join(
                                                                                 state['chat_history'][-5:]),
                                                                             extracted_entity=str(
                                                                                 state['node_data']))}]

In [36]:
Output = {'Action': 'Response',
 'attribute': "I still need the doctor's NPI. Please provide it.",
 'extracted_entity': {'provider_name': 'Emily Carter',
  'npi_number': '',
  'patient_name': '',
  'patient_dob': ''}}

In [37]:
#For Travel 

Output = {
    "Action": "Response",
    "attribute": "I still need your booking reference number. Please provide it.",
    "extracted_entity": {
        "name": "James",
        "email_id": "jms@gmail.com",
        "password": ""
    }
}

In [39]:
import json
data_final = {'data':[]}
data = {
    'prompt': [messages],
    'output': [Output]
}

NameError: name 'messages' is not defined

In [39]:
data_final['data'].append(data)

In [40]:
print(data)

{'prompt': [[{'role': 'system', 'content': '\n{\n    "input": [\n        {\n            "role": "system",\n            "content": "You are the verification block of chatbot Marvin, your task is to gather information about the user\'s name, email ID, and password, and verify the information using the provided tool. If some information is missing, then you should ask follow-up questions."\n        },\n        {\n            "role": "user",\n            "content": "<user_input>"\n        },\n        {\n            "role": "verify_user_details",\n            "parameters": {\n                "name": "",\n                "email": "",\n                "password": ""\n            }\n            "content": {\n            "Action": "Response",\n            "attribute": "Thank you James. You are logged in now! How can I assist you today with your recent bookings or new ticket bookings?"\n            }\n        }\n    ],\n    "Output": {\n        "Action": "Response",\n        "attribute": "Thank 

### Samples

In [15]:
#1
destination_suggestions_prompt='''
{
    "input": [
        {
            "role": "system",
            "content": "You are the destination suggestions block of chatbot Marvin. Your task is to suggest travel destinations based on the user's preferences. Gather information about the user's interests, budget, and preferred travel dates. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "<user_input>Please suggest me with some destinations."
        },
        {
            "role": "assistant",
            "parameters": {
                "interests": "",
                "budget": "",
                "travel_dates": ""
            },
            "content": {
                "Action": "Response",
                "attribute": "Based on your interests in beaches and a budget of $2000, I suggest visiting Bali in June. It's a beautiful destination with stunning beaches and affordable accommodations."
            }
        }
    ],
    "Output": {
        "Action": "Response",
        "attribute": "Based on your interests in beaches and a budget of $2000, I suggest visiting Bali in June. It's a beautiful destination with stunning beaches and affordable accommodations.",
        "extracted_entity": {
            "interests": "beaches",
            "budget": "$2000",
            "travel_dates": "June"
        }
    }
}
'''


In [14]:
destination_suggestions_prompt

'\n{\n    "input": [\n        {\n            "role": "system",\n            "content": "You are the destination suggestions block of chatbot Marvin. Your task is to suggest travel destinations based on the user\'s preferences. Gather information about the user\'s interests, budget, and preferred travel dates. If some information is missing, ask follow-up questions to get a complete picture."\n        },\n        {\n            "role": "user",\n            "content": "<user_input>Please suggest me with some destinations."\n        },\n        {\n            "role": "destination_preferences",\n            "parameters": {\n                "interests": "",\n                "budget": "",\n                "travel_dates": ""\n            },\n            "content": {\n                "Action": "Response",\n                "attribute": "Based on your interests in beaches and a budget of $2000, I suggest visiting Bali in June. It\'s a beautiful destination with stunning beaches and affordable ac

In [19]:
#####
ticket_booking= {
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_ticket_status\", \"description\": \"Check the status of a ticket\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The ticket number to check the status for\"}}, \"required\": [\"ticket_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me book a movie ticket? </user_input>"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "I'm sorry, but as an AI, I don't have the capability to perform external tasks such as movie ticket booking. My current function allows me to book travel ticket. If you need assistance with that, please provide your ticket number."
        }
    ]
}


In [26]:
ticket_booking

{'input': [{'role': 'system',
   'content': 'You are a helpful assistant with access to functions and tools. Use them if required.\n[{"name": "check_ticket_status", "description": "Check the status of a ticket", "parameters": {"type": "object", "properties": {"ticket_number": {"type": "string", "description": "The ticket number to check the status for"}}, "required": ["ticket_number"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     "Action":<either Response or Tool call as per the output>\n     "attribute":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n'},
  {'role': 'user',
   'content': '<user_input> Can you help me book a movie ticket? </user_input>'}],
 'output': [{'Action': 'Response',
   'attribute': "I'm sorry, but as an AI, I don't have the capability to p

In [35]:
ticket_booking2= {
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_ticket_status\", \"description\": \"Check the status of a ticket\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The ticket number to check the status for\"}}, \"required\": [\"ticket_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me book a flight ticket? </user_input>"
        },
        {
            "role": "assistant",
            "content": "Yes, Sure! Please provide me with Departure Location your and destination "
        },
        {
            "role": "user",
            "content": "<user_input> I want to travel to Bali from New Delhi </user_input>"
        },
        {
            "role": "assistant",
            "content": "Please confirm your booking by giving a Yes/No !"
        },
        {
            "role": "user",
            "content": "<user_input> Yes! Proceed </user_input>"
        },
        {
            "role": "assistant",
            "content": "Congratulations! Your flight ticket from New Delhi to Bali is confirmed!"
        }
        
    ],
     "output": [
        {
            "Action": "Response",
            "attribute": {
                "name": "booking_init",
                "arguments": {
                    "booking_number": "ABC123",
                    "Departure From": "New Delhi",
                    "Destination": "Bali",
                    "Status": "Booking is Confirmed :) "
                }
            }
        }
    ]
}

In [38]:
hotel_suggestions={
    "input": [
        {
            "role": "system",
            "content": "You are the hotel suggestions block of chatbot Marvin. Your task is to suggest hotels based on the user's preferences. Gather information about the user's destination, budget, and preferred amenities. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "<user_input>Please suggest me with some hotels.</user_input>"
        },
        {
            "role": "assistant",
            "content": "Sure! Could you please provide me with your destination, budget, and preferred amenities?"
        },
        {
            "role": "user",
            "content": "<user_input>I'm looking for a beachfront hotel in Bali with a budget of $150 per night.</user_input>"
        },
        {
            "role": "assistant",
            "content": "Based on your preference for a beachfront hotel in Bali with a budget of $150 per night, I suggest staying at the Bali Beach Resort. It offers stunning views, free breakfast, and a spa."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "destination": "Bali",
                "budget": "$150 per night",
                "amenities": "beachfront, free breakfast, spa"
            }
        }
    ]
}


In [40]:
my_ticket_cancellation={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"cancel_ticket\", \"description\": \"Cancel a ticket\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The ticket number to cancel\"}}, \"required\": [\"ticket_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me cancel my flight ticket? </user_input>"
        },
        {
            "role": "assistant",
            "content": "Yes, Sure! Please provide me with your ticket number."
        },
        {
            "role": "user",
            "content": "<user_input> My ticket number is ABC123 </user_input>"
        },
        {
            "role": "assistant",
            "content": "Please confirm your cancellation by giving a Yes/No!"
        },
        {
            "role": "user",
            "content": "<user_input> Yes! Proceed </user_input>"
        },
        {
            "role": "assistant",
            "content": "Your flight ticket with ticket number ABC123 has been successfully cancelled."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "name": "cancellation_init",
                "arguments": {
                    "ticket_number": "ABC123",
                    "Status": "Cancellation is Confirmed :)"
                }
            }
        }
    ]
}


In [41]:
refund_req={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"process_refund\", \"description\": \"Process a refund\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The ticket number for the refund\"}}, \"required\": [\"ticket_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me with a refund for my flight ticket? </user_input>"
        },
        {
            "role": "assistant",
            "content": "Yes, Sure! Please provide me with your ticket number."
        },
        {
            "role": "user",
            "content": "<user_input> My ticket number is ABC123 </user_input>"
        },
        
        {
            "role": "assistant",
            "content": "Your refund request for ticket number ABC123 has been processed. The refund will be reflected in your account within 48 hours."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "name": "refund_init",
                "arguments": {
                    "ticket_number": "ABC123",
                    "Status": "Refund is processed and will be reflected in 48 hours :)"
                }
            }
        }
    ]
}


In [38]:
hotel_suggestions={
    "input": [
        {
            "role": "system",
            "content": "You are the hotel suggestions block of chatbot Marvin. Your task is to suggest hotels based on the user's preferences. Gather information about the user's destination, budget, and preferred amenities. If some information is missing, ask follow-up questions to get a complete picture."
        },
        {
            "role": "user",
            "content": "<user_input>Please suggest me with some hotels.</user_input>"
        },
        {
            "role": "assistant",
            "content": "Sure! Could you please provide me with your destination, budget, and preferred amenities?"
        },
        {
            "role": "user",
            "content": "<user_input>I'm looking for a beachfront hotel in Bali with a budget of $150 per night.</user_input>"
        },
        {
            "role": "assistant",
            "content": "Based on your preference for a beachfront hotel in Bali with a budget of $150 per night, I suggest staying at the Bali Beach Resort. It offers stunning views, free breakfast, and a spa."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "destination": "Bali",
                "budget": "$150 per night",
                "amenities": "beachfront, free breakfast, spa"
            }
        }
    ]
}


In [42]:
train_ticket_status_check={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_train_ticket_status\", \"description\": \"Check the status of a train ticket\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The train ticket number to check the status for\"}}, \"required\": [\"ticket_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me check the status of my train ticket? </user_input>"
        },
        {
            "role": "assistant",
            "content": "Yes, Sure! Please provide me with your train ticket number."
        },
        {
            "role": "user",
            "content": "<user_input> My train ticket number is TKT456 </user_input>"
        },
        {
            "role": "assistant",
            "content": "Please confirm your request by giving a Yes/No!"
        },
        {
            "role": "user",
            "content": "<user_input> Yes! Proceed </user_input>"
        },
        {
            "role": "assistant",
            "content": "Your train ticket with ticket number TKT456 is confirmed."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "name": "train_ticket_status_check",
                "arguments": {
                    "ticket_number": "TKT456",
                    "Status": "Ticket is confirmed :)"
                }
            }
        }
    ]
}


In [36]:
my_status_check= {
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_booking_status\", \"description\": \"Check the status of a travel booking\", \"parameters\": {\"type\": \"object\", \"properties\": {\"booking_number\": {\"type\": \"string\", \"description\": \"The booking number to check the status for\"}}, \"required\": [\"booking_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Hi, I need to check the status of my travel booking. </user_input>"
        },
        {
            "role": "assistant",
            "content": {
                "Action": "Response",
                "attribute": "Sure, I can help with that. Could you please provide me with your booking number?"
            }
        },
        {
            "role": "user",
            "content": "<user_input> Yes, it's ABC123. </user_input>"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "name": "booking_status",
                "arguments": {
                    "booking_number": "ABC123",
                    "From": "New Delhi",
                    "Destination": "Bali",
                    "Status": "Booking is Confirmed :) "
                }
            }
        }
    ]
}


In [37]:
my_status_check

{'input': [{'role': 'system',
   'content': 'You are a helpful assistant with access to functions and tools. Use them if required.\n[{"name": "check_booking_status", "description": "Check the status of a travel booking", "parameters": {"type": "object", "properties": {"booking_number": {"type": "string", "description": "The booking number to check the status for"}}, "required": ["booking_number"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     "Action":<either Response or Tool call as per the output>\n     "attribute":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n'},
  {'role': 'user',
   'content': '<user_input> Hi, I need to check the status of my travel booking. </user_input>'},
  {'role': 'assistant',
   'content': {'Action': 'Response',
    'attribute': 'Sur

In [43]:
date_of_travel_change={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"change_travel_date\", \"description\": \"Change the travel date of a ticket\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The ticket number to change the travel date for\"}, \"new_travel_date\": {\"type\": \"string\", \"description\": \"The new travel date\"}}, \"required\": [\"ticket_number\", \"new_travel_date\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me change the travel date of my ticket? </user_input>"
        },
        {
            "role": "assistant",
            "content": "Yes, Sure! Please provide me with your ticket number and the new travel date."
        },
        {
            "role": "user",
            "content": "<user_input> My ticket number is TKT789 and I want to change the travel date to April 15th. </user_input>"
        },
        {
            "role": "assistant",
            "content": "Please confirm your request to change the travel date from April 10th to April 15th by giving a Yes/No!"
        },
        {
            "role": "user",
            "content": "<user_input> Yes! Proceed </user_input>"
        },
        {
            "role": "assistant",
            "content": "Your travel date has been successfully changed to April 15th for ticket number TKT789."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": {
                "name": "travel_date_change",
                "arguments": {
                    "ticket_number": "TKT789",
                    "new_travel_date": "April 15th",
                    "Status": "Travel date has been successfully changed :)"
                }
            }
        }
    ]
}


In [44]:
book_my_show={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_ticket_status\", \"description\": \"Check the status of a ticket\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ticket_number\": {\"type\": \"string\", \"description\": \"The ticket number to check the status for\"}}, \"required\": [\"ticket_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me book a concert ticket? </user_input>"
        },
        {
            "role": "assistant",
            "content": "I'm sorry, but I can only assist with travel-related queries. Is there anything else you need help with regarding your travel plans?"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "I'm sorry, but I can only assist with travel-related queries. Is there anything else you need help with regarding your travel plans?"
        }
    ]
}


In [45]:
basic_plesantries={
    "input": [
        {
            "role": "system",
            "content": "You are a friendly assistant. Your task is to greet the user warmly and ask how you can assist them."
        },
        {
            "role": "user",
            "content": "<user_input>Hello!</user_input>"
        },
        {
            "role": "assistant",
            "content": "Hi there! How can I assist you today?"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "Hi there! How can I assist you today?"
        }
    ]
}


In [46]:
goodbye_prompting={
    "input": [
        {
            "role": "system",
            "content": "You are a friendly assistant. Your task is to say goodbye to the user warmly and wish them well."
        },
        {
            "role": "user",
            "content": "<user_input>Goodbye!</user_input>"
        },
        {
            "role": "assistant",
            "content": "Goodbye! Have a great day!"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "Goodbye! Have a great day!"
        }
    ]
}


In [47]:
thankyou_prompt={
    "input": [
        {
            "role": "system",
            "content": "You are a friendly assistant. Your task is to thank the user for their interaction and offer assistance if needed."
        },
        {
            "role": "user",
            "content": "<user_input>Thank you!</user_input>"
        },
        {
            "role": "assistant",
            "content": "You're welcome! If you need any further assistance, feel free to ask."
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "You're welcome! If you need any further assistance, feel free to ask."
        }
    ]
}


In [48]:
out_of_scope={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant. Your task is to politely inform the user that their request is out of scope and redirect them to relevant assistance."
        },
        {
            "role": "user",
            "content": "<user_input>Can you help me with my homework?</user_input>"
        },
        {
            "role": "assistant",
            "content": "I'm sorry, but I'm currently unable to assist with homework. Is there anything else I can help you with?"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "I'm sorry, but I'm currently unable to assist with homework. Is there anything else I can help you with?"
        }
    ]
}


In [49]:
out_of_scope2={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant. Your task is to politely inform the user that their request is out of scope and suggest alternative resources."
        },
        {
            "role": "user",
            "content": "<user_input>Can you give me medical advice?</user_input>"
        },
        {
            "role": "assistant",
            "content": "I'm not qualified to provide medical advice. I recommend consulting a healthcare professional for assistance. Is there anything else I can help you with?"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "I'm not qualified to provide medical advice. I recommend consulting a healthcare professional for assistance. Is there anything else I can help you with?"
        }
    ]
}


In [55]:


travel_insurance={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"purchase_travel_insurance\", \"description\": \"Purchase travel insurance for the user\", \"parameters\": {\"type\": \"object\", \"properties\": {\"destination\": {\"type\": \"string\", \"description\": \"The travel destination\"}, \"start_date\": {\"type\": \"string\", \"description\": \"The start date of the insurance\"}, \"end_date\": {\"type\": \"string\", \"description\": \"The end date of the insurance\"}, \"number_of_people\": {\"type\": \"integer\", \"description\": \"The number of people for whom the insurance is needed\"}}, \"required\": [\"destination\", \"start_date\", \"end_date\", \"number_of_people\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n {\n \"Action\":\n \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "Hi, I need travel insurance."
        },
        {
            "role": "assistant",
            "content": {
                "Action": "Response",
                "attribute": "Sure, I can help with that. Could you please provide me with your travel destination, start date, end date of the insurance, and the number of people for whom the insurance is needed?"
            }
        },
        {
            "role": "user",
            "content": "I need travel insurance for my trip to Japan from April 5th to April 20th for 2 people."
        }
    ],
    "output": [
        {
            "Action": "Tool call to travel_insurance_payment",
            "attribute": {
                "name": "purchase_travel_insurance",
                "arguments": {
                    "destination": "Japan",
                    "start_date": "April 5th",
                    "end_date": "April 20th",
                    "number_of_people": 2
                }
            }
        }
    ]
}

        

In [54]:
train_ticket_booking_pay={
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"book_train_ticket\", \"description\": \"Book a train ticket for the user\", \"parameters\": {\"type\": \"object\", \"properties\": {\"departure_station\": {\"type\": \"string\", \"description\": \"The departure station\"}, \"destination_station\": {\"type\": \"string\", \"description\": \"The destination station\"}, \"travel_date\": {\"type\": \"string\", \"description\": \"The travel date\"}, \"number_of_passengers\": {\"type\": \"integer\", \"description\": \"The number of passengers\"}}, \"required\": [\"departure_station\", \"destination_station\", \"travel_date\", \"number_of_passengers\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n {\n \"Action\":\n \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "Hi, I need to book a train ticket."
        },
        {
            "role": "assistant",
            "content": {
                "Action": "Response",
                "attribute": "Sure, I can help with that. Could you please provide me with the departure station, destination station, travel date, and the number of passengers?"
            }
        },
        {
            "role": "user",
            "content": "I want to book a train ticket from New York to Washington D.C. on May 15th for 2 passengers."
        }
    ],
    "output": [
        {
            "Action": "Tool call to payment_gateway",
            "attribute": {
                "name": "book_train_ticket",
                "arguments": {
                    "departure_station": "New York",
                    "destination_station": "Washington D.C.",
                    "travel_date": "May 15th",
                    "number_of_passengers": "2",
                    "cost" : "$150"
                }
            }
        }
    ]
}


In [58]:
flight_booking_identification={
    "input": [
        {
            "role": "system",
            "content": "Your role is to identify the user's intent and guide the conversation towards one of the supported travel-related processes. You are the *first* step in the conversation flow.\n\n**Supported Processes:**\n\n1.  Flight Booking: The user wants to book a flight (e.g., \"book a flight,\" \"reserve a plane ticket\").\n2.  Hotel Booking: The user wants to book a hotel (e.g., \"book a hotel,\" \"reserve a room\").\n3.  Car Rental: The user wants to rent a car (e.g., \"rent a car,\" \"book a rental car\").\n4.  Travel Insurance: The user wants to purchase travel insurance (e.g., \"buy travel insurance,\" \"get travel coverage\").\n\n**Handling \"Other\" Intents:**\n\nIf the user's intent does *not* clearly fall into one of the supported categories, classify it as **`Other`**. This includes:\n\n*   **Greetings:** (e.g., \"Hi,\" \"Hello,\" \"Good morning\")\n*   **Thanks/Gratitude:** (e.g., \"Thank you,\" \"Thanks,\" \"Appreciate it\")\n*   **General Inquiries/Small Talk:** Questions or statements not directly related to the supported processes.\n*   **Unclear Requests:**  If the user's input is ambiguous or nonsensical.\n\nFor the **`Other`** intent:\n\n*   generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported travel-related processes.\n\n**Input:**\n\n*   **`user_input`:** The user's most recent message (string).\n*   **`chat_history`:** The full conversation history (list of strings, alternating between user and assistant messages).  Use the `chat_history` to help determine the intent, especially if the `user_input` is short or ambiguous.\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n{\n    \"Action\":\"<Intent>\",\n    \"attribute\":\"<Response or null>\"\n}\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order."
        },
        {
            "role": "user",
            "content": "<user_input>\nuser: I need to book a flight to Paris. \n</user_input>\n\n<chat_history>\nassistant: Hello, my name is Marvin. How can I assist you today? \nuser: Hi. \nuser: I need to travel soon. \nassistant: Where would you like to go? \nuser: Paris. \n</chat_history>"
        }
    ],
    "output": [
        {
            "Action": "Flight Booking",
            "attribute": None
        }
    ]
}


In [59]:
hotel_booking_identification={
    "input": [
        {
            "role": "system",
            "content": "Your role is to identify the user's intent and guide the conversation towards one of the supported travel-related processes. You are the *first* step in the conversation flow.\n\n**Supported Processes:**\n\n1.  Flight Booking: The user wants to book a flight (e.g., \"book a flight,\" \"reserve a plane ticket\").\n2.  Hotel Booking: The user wants to book a hotel (e.g., \"book a hotel,\" \"reserve a room\").\n3.  Car Rental: The user wants to rent a car (e.g., \"rent a car,\" \"book a rental car\").\n4.  Travel Insurance: The user wants to purchase travel insurance (e.g., \"buy travel insurance,\" \"get travel coverage\").\n\n**Handling \"Other\" Intents:**\n\nIf the user's intent does *not* clearly fall into one of the supported categories, classify it as **`Other`**. This includes:\n\n*   **Greetings:** (e.g., \"Hi,\" \"Hello,\" \"Good morning\")\n*   **Thanks/Gratitude:** (e.g., \"Thank you,\" \"Thanks,\" \"Appreciate it\")\n*   **General Inquiries/Small Talk:** Questions or statements not directly related to the supported processes.\n*   **Unclear Requests:**  If the user's input is ambiguous or nonsensical.\n\nFor the **`Other`** intent:\n\n*   generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported travel-related processes.\n\n**Input:**\n\n*   **`user_input`:** The user's most recent message (string).\n*   **`chat_history`:** The full conversation history (list of strings, alternating between user and assistant messages).  Use the `chat_history` to help determine the intent, especially if the `user_input` is short or ambiguous.\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n{\n    \"Action\":\"<Intent>\",\n    \"attribute\":\"<Response or null>\"\n}\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order."
        },
        {
            "role": "user",
            "content": "<user_input>\nuser: I need to book a hotel in New York. \n</user_input>\n\n<chat_history>\nassistant: Hello, my name is Marvin. How can I assist you today? \nuser: Hi. \nuser: I need accommodation for my trip. \nassistant: Where are you traveling to? \nuser: New York. \n</chat_history>"
        }
    ],
    "output": [
        {
            "Action": "Hotel Booking",
            "attribute": None
        }
    ]
}


In [1]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://cxaicoe-openai.openai.azure.com/'
os.environ["OPENAI_API_KEY"] = 'f0dd5fb480d74684832ff376296b730b'

llm = AzureChatOpenAI(**{'deployment_name':'cx-aicoe-gpt4','openai_api_version':'2024-09-01-preview'},temperature=0.1)
import json 
import random
from langchain_core.prompts import PromptTemplate
import re

with open('sample_inputs_Final.json','r') as file:
    data = json.load(file)[:20]

prompt_inputs = [record['input'] for record in data] 
print("test")

def balance_brackets(json_string):
    # Stack to track opening brackets and their positions
    stack = []
    
    # Convert the string into a list of characters to modify it
    json_chars = list(json_string)
    
    # Iterate over the string to track open and close brackets
    for idx, char in enumerate(json_chars):
        if char == '{' or char == '[':
            stack.append((char, idx))  # Push opening brackets and their indices to the stack
        elif char == '}' or char == ']':
            if stack:
                last_open, last_idx = stack.pop()  # Pop the last opening bracket
                # If it's a mismatched pair, fix it
                if (char == '}' and last_open != '{') or (char == ']' and last_open != '['):
                    json_chars[last_idx] = ')'  # Replace last unmatched open bracket to correct type
                    stack.append((char, idx))   # Push this closing bracket back to stack
            else:
                # If it's an unmatched closing bracket, ignore it (but this could be handled as error)
                pass

    # Handle any unmatched opening brackets by adding the corresponding closing ones at the end
    while stack:
        last_open, last_idx = stack.pop()
        if last_open == '{':
            json_chars.append('}')  # Add closing brace at the end
        elif last_open == '[':
            json_chars.append(']')  # Add closing square bracket at the end
    
    # Return the balanced string
    return ''.join(json_chars)

output_format = '''{
        "user": "<Predicted user_input>",
        "Action": "Other",
        "attribute": "<Response as per user input in a string>"
        }'''
        
c=0
all_responses = []
for i in range(len(prompt_inputs)):
    chat_history_match = re.search(r'<chat_history>(.*?)</chat_history>', prompt_inputs[i], re.DOTALL)
    user_inputs_match = re.search(r'<user_input>(.*?)</user_input>', prompt_inputs[i], re.DOTALL)
    chat_history_raw = chat_history_match.group(1).strip() if chat_history_match else ""
    user_inputs_match = user_inputs_match.group(1).strip() if user_inputs_match else ""
    prompt_message = f''' 
    For the provided prompt, generate **15** alternative user inputs for each of the following scenarios:

    1. **Out-of-scope**: These are user inputs that are unrelated to the system's capabilities (e.g., if the system is designed for energy-related inquiries, user inputs like "Can you order me a pizza?" are out-of-scope).
    2. **Greet**: These are user inputs where the user greets the assistant (e.g., "Hi", "Hello", "Good morning").
    3. **Bye**: These are user inputs where the user says goodbye or ends the interaction (e.g., "Thanks", "I'm done", "Goodbye").
    4. **Say again**: These are user inputs where the user asks the assistant to repeat the last statement (e.g., "Come again", "What did you say?", "Can you repeat that?").
    5. **Hold on**: These are user inputs where the user asks the assistant to hold on for a moment (e.g., "Hold on a second", "Give me a moment", "Let me gather that information").

    For each user input, predict the corresponding **Action** and **Attribute** taking {chat_history_raw} into consideration.

    prompt: {prompt_inputs[i]}

    Output:
    [
        {output_format}
    ]

    Note: Please ensure that user inputs are diverse, realistic, and not repetitive. Return the output in the specified JSON format.     
    '''
    messages = [
                SystemMessage(content=prompt_message)
            ]
    try:
        response = llm.invoke(messages).content
        # print(response)
        response = balance_brackets(response)
        cleaned_response = re.sub(r'^```json\n|\n```$', '', response).strip()
        response_json = json.loads(cleaned_response)
        #print(response_json)
        user_inputs = [item['user'] for item in response_json]
        prompt_inputs_replaced = [prompt_inputs[i].replace(user_inputs_match, f"{user_input}") for user_input in user_inputs]
        print(prompt_inputs_replaced)
        datapoint_jsons = [{"input":prompt_input ,"output":[{"Action":"Other", "attribute":output_json['attribute']}]}
                           for (output_json, prompt_input) in zip(response_json, prompt_inputs_replaced)]
        all_responses += datapoint_jsons # Add the new responses to the overall list
    except json.JSONDecodeError:
        print(f"Error parsing response for prompt {i}.")
    c += 1
    print(c)

with open('responses_first_20.json', 'w') as outfile:
    json.dump(all_responses, outfile, indent=4)


FileNotFoundError: [Errno 2] No such file or directory: 'sample_inputs_Final.json'

In [3]:
#Example 1: Tool call 
{
        "input": [
            {
                "role": "system",
                "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"generate_qr_code\", \"description\": \"Generate a QR code for a given input\", \"parameters\": {\"type\": \"object\", \"properties\": {\"data\": {\"type\": \"string\", \"description\": \"The data to encode in the QR code\"}, \"size\": {\"type\": \"integer\", \"description\": \"The size of the QR code\"}}, \"required\": [\"data\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
            },
            {
                "role": "user",
                "content": "<user_input> Hi, I need a QR code for my website. </user_input>"
            },
            {
                "role": "assistant",
                "content": {
                    "Action": "Response",
                    "attribute": "Sure, I can help with that. Could you please provide me with the URL of your website?"
                }
            },
            {
                "role": "user",
                "content": "<user_input> Yes, it's www.mywebsite.com. </user_input>"
            }
        ],
        "output": [
            {
                "Action": "Tool call",
                "attribute": {
                    "name": "generate_qr_code",
                    "arguments": {
                        "data": "www.mywebsite.com",
                        "size": 200
                    }
                }
            }
        ]
    }


#Example 2: Response 
{
        "input": [
            {
                "role": "system",
                "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"create_note\", \"description\": \"Create a new note\", \"parameters\": {\"type\": \"object\", \"properties\": {\"title\": {\"type\": \"string\", \"description\": \"The title of the note\"}, \"content\": {\"type\": \"string\", \"description\": \"The content of the note\"}}, \"required\": [\"title\", \"content\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
            },
            {
                "role": "user",
                "content": "<user_input> Can you order a pizza for me? </user_input>"
            }
        ],
        "output": [
            {
                "Action": "Response",
                "attribute": "I'm sorry, but as an AI, I don't have the capability to perform external tasks such as ordering a pizza. My current function allows me to create a note for you. If you need to remember to order a pizza later, I can help you create a note for that."
            }
        ]
    }


#Example 3: Process Identification
{
        "input": [
	    {
		"role": "system",
		"content: "Your role is to identify the user's intent and guide the conversation towards one of the supported travel-related processes. You are the *first* step in the conversation flow.\n\n**Supported Processes:**\n\n1.  Booking: The user is making a new booking (e.g., \"make a booking,\" \"book an appointment,\" \"schedule my meeting\").\n2.  Rebooking: The user is rescheduling an existing booking (e.g., \"rebook my flight,\" \"change my appointment time\").\n3.  Cancellation: The user wants to cancel a service, booking, or other request (e.g., \"cancel my booking,\" \"I want to cancel the service\").\n4.  Modification: The user wants to make changes to an existing booking or service (e.g., \"change my reservation,\" \"update my contact information\").\n5.  extra_services: The user is requesting, upgrading, or adding additional services or features to an existing booking (e.g., \"add a meal,\" \"upgrade my seat,\" \"request early check-in\")\n\n**Handling \"Other\" Intents:**\n\nIf the user's intent does *not* clearly fall into one of the supported categories, classify it as **`Other`**. This includes:\n\n*   **Greetings:** (e.g., \"Hi,\" \"Hello,\" \"Good morning\")\n*   **Thanks/Gratitude:** (e.g., \"Thank you,\" \"Thanks,\" \"Appreciate it\")\n*   **General Inquiries/Small Talk:** Questions or statements not directly related to the supported processes.\n*   **Unclear Requests:**  If the user's input is ambiguous or nonsensical.\n\nFor the **`Other`** intent:\n\n*   generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported pharmacy-related processes.\n\n**Input:**\n\n*   **`user_input`:** The user's most recent message (string).\n*   **`chat_history`:** The full conversation history (list of strings, alternating between user and assistant messages).  Use the `chat_history` to help determine the intent, especially if the `user_input` is short or ambiguous.\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n{\n    \"Action\":\"<Intent>\",\n    \"attribute\":\"<Response or null>\"\n}\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order."
	},
	{
	     "role": "user",
	     "content": "<user_input>\nuser: this was a Refundable ticket i believe. \n</user_input>\n\n<chat_history>\nassistant: Hello, null, my name is Jeni. To assist you, I will need your order number. \nuser: Hi. \nuser: Order no : 89sjb5. \nassistant: How can I help you? \nuser: i i am trying to cancel 2 tickets out of 4 booked. \nuser: Can you please help with that. \nassistant: No Refund will be possible for cancellation. \n</chat_history>"
	}
    ],
        "output": [
            {
                "Action": "Cancellation",
                "attribute": null
            }
        ]
    }

SyntaxError: invalid syntax (1293042250.py, line 65)

In [ ]:
#Case 1: Tool call
[
	{
		"Action":"Tool call", 
		"attribute":{
			"name":<tool name>, 
			"arguments":<tool arguments in the form of a DICT>
		}
	}
]


#Case 2: Response
[
	{
		"Action":"Response", 
		"attribute":<response string>
	}
]


Case 3: Process Detection 
[
	{
		"Action":<process name>, 
		"attribute":null
	}
]


Case 4: No applicable process found
[
	{
		"Action":"Other", 
		"attribute":<response string>
	}
]				